# CHIKV_Mac HIPPO

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import hippo
from tqdm import tqdm
import molparse as mp
from posebutcher import PoseButcher
from mlog import setup_logger
logger = setup_logger('CHIKV_Mac_HIPPO')

### Start from a fresh database

In [3]:
!rm -v CHIKV_master.sqlite

CHIKV_master.sqlite


In [4]:
load_fresh = True

In [28]:
animal = hippo.HIPPO('CHIKV', 'CHIKV_master.sqlite')

Creating HIPPO animal
name = CHIKV
db_path = CHIKV_master.sqlite
DEBUG: hippo3.Database.__init__()
DEBUG: Database.path = CHIKV_master.sqlite
DEBUG: hippo3.Database.connect()
DEBUG: sqlite3.version='2.6.0'
 SUCCESS  Database connected @ CHIKV_master.sqlite!
 SUCCESS  Initialised animal HIPPO("CHIKV")!


### Load the native substrate

In [6]:
if load_fresh:
    pdb_6vuq = mp.parse('../6vuq_B.pdb')
    mol = pdb_6vuq['rLIG'][0].rdkit_mol
    animal.register_target(name='CHIKV_Mac')
    comp = animal.register_compound(smiles=mp.rdkit.mol_to_smiles(mol))
    animal.register_pose(alias='6vuq APR (B)', compound=comp, target='CHIKV_Mac', path='../6vuq_B.pdb')

parsing ../6vuq_B.pdb ...                 Done.


## Add hits from Fragalysis

In [7]:
if load_fresh:
    animal.add_hits("CHIKV_Mac", '../metadata.csv', '../aligned_files')

 Warning  Skipping existing target with name='CHIKV_Mac'
aligned_directory = ../aligned_files


324it [01:14,  4.37it/s]

#directories parsed = 323
#compounds registered = 89
#poses registered = 323


In [8]:
animal.poses.interactive()

Output()

## Tag with PoseButcher

In [31]:
butcher_6vuq = PoseButcher.from_directory('../butcher_6VUQ')

 DISK  Reading ../butcher_6VUQ/butcher_6VUQ.json
 DISK  Reading ../butcher_6VUQ/protein.pdb
 DISK  Writing _butcher_protein.pdb
 DISK  Reading ../butcher_6VUQ/_protein_hull.ply
 DISK  Reading ../butcher_6VUQ/_protein_mesh.ply
 DISK  Reading ../butcher_6VUQ/pocket_adenine (A).ply
 DISK  Reading ../butcher_6VUQ/pocket_ribose (A).ply
 DISK  Reading ../butcher_6VUQ/pocket_phosphate (A).ply
 DISK  Reading ../butcher_6VUQ/pocket_oxyanion (A).ply
 DISK  Reading ../butcher_6VUQ/pocket_arginine (A).ply
 DISK  Reading ../butcher_6VUQ/pocket_adenine (B).ply
 DISK  Reading ../butcher_6VUQ/pocket_ribose (B).ply
 DISK  Reading ../butcher_6VUQ/pocket_phosphate (B).ply
 DISK  Reading ../butcher_6VUQ/pocket_oxyanion (B).ply
 DISK  Reading ../butcher_6VUQ/pocket_arginine (B).ply
 DISK  Reading ../butcher_6VUQ/pocket_adenine (C).ply
 DISK  Reading ../butcher_6VUQ/pocket_ribose (C).ply
 DISK  Reading ../butcher_6VUQ/pocket_phosphate (C).ply
 DISK  Reading ../butcher_6VUQ/pocket_oxyanion (C).ply
 DISK  Rea

In [10]:
def tags_from_butcher(pose, butcher=None):
    
    if not butcher:
        print(pose.path)
        
        pockets = {
            'adenine': dict(type='sphere', atoms=['ARG 144 CG', 'VAL 33 CG2'], radius=4.5),
            'ribose': dict(type='sphere', atoms=['TYR 114 CZ', 'ASP 31 CA', 'TYR 114 CG', 'GLY 30 C'], radius=4),
            'phosphate': dict(type='sphere', atoms=['SER 110 N', 'GLY 32 N', 'THR 111 N'], radius=4),
            'oxyanion': dict(type='sphere', atoms=['THR 111 OG1', 'ASP 145 N'], radius=4),
            'arginine': dict(type='sphere', atoms=['ARG 144 NE', 'ASP 10 N', 'PRO 41 CB']),
        }
        
        butcher = PoseButcher.from_directory('butcher_6vuq')
        butcher.protein = pose.path
        
        # butcher = PoseButcher(pose.path, 
        #                       pockets=pockets, 
        #                       pocket_chains='ABCD', 
        #                       pocket_clip=True, 
        #                       pocket_clip_hull=False, 
        #                       pocket_clip_protein=False,
        #                      )
    
    tags = butcher.tag(pose.mol, pockets_only=True)
    
    tags = set([f'butcher: {t.split()[0]}' for t in tags])
    
    for t in tags:
        pose.tags.add(t)
        # (f'added {t}')
        
    return butcher

In [11]:
if load_fresh:
    tags_from_butcher(animal.poses[1])

    pose = animal.poses[2]

    for pose in tqdm(animal.poses[2:]):
        butcher = tags_from_butcher(pose)

/Users/tfb64483/Data/CHIKV_Mac/6vuq_B.pdb
 DISK  Reading butcher_6vuq/butcher_6VUQ.json
 DISK  Reading butcher_6vuq/protein.pdb
 DISK  Writing _butcher_protein.pdb
 DISK  Reading butcher_6vuq/_protein_hull.ply
 DISK  Reading butcher_6vuq/_protein_mesh.ply
 DISK  Reading butcher_6vuq/pocket_adenine (A).ply
 DISK  Reading butcher_6vuq/pocket_ribose (A).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (A).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (A).ply
 DISK  Reading butcher_6vuq/pocket_arginine (A).ply
 DISK  Reading butcher_6vuq/pocket_adenine (B).ply
 DISK  Reading butcher_6vuq/pocket_ribose (B).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (B).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (B).ply
 DISK  Reading butcher_6vuq/pocket_arginine (B).ply
 DISK  Reading butcher_6vuq/pocket_adenine (C).ply
 DISK  Reading butcher_6vuq/pocket_ribose (C).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (C).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (C).ply
 DISK  Reading butcher

  0%|                                                                                                                                             | 0/322 [00:00<?, ?it/s]

/Users/tfb64483/Data/CHIKV_Mac/aligned_files/cx0312c/cx0312c.pdb
 DISK  Reading butcher_6vuq/butcher_6VUQ.json
 DISK  Reading butcher_6vuq/protein.pdb
 DISK  Writing _butcher_protein.pdb
 DISK  Reading butcher_6vuq/_protein_hull.ply
 DISK  Reading butcher_6vuq/_protein_mesh.ply
 DISK  Reading butcher_6vuq/pocket_adenine (A).ply
 DISK  Reading butcher_6vuq/pocket_ribose (A).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (A).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (A).ply
 DISK  Reading butcher_6vuq/pocket_arginine (A).ply
 DISK  Reading butcher_6vuq/pocket_adenine (B).ply
 DISK  Reading butcher_6vuq/pocket_ribose (B).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (B).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (B).ply
 DISK  Reading butcher_6vuq/pocket_arginine (B).ply
 DISK  Reading butcher_6vuq/pocket_adenine (C).ply
 DISK  Reading butcher_6vuq/pocket_ribose (C).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (C).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (C).ply

  0%|▍                                                                                                                                    | 1/322 [00:00<04:56,  1.08it/s]

/Users/tfb64483/Data/CHIKV_Mac/aligned_files/cx1108b/cx1108b_hippo.pdb
 DISK  Reading butcher_6vuq/butcher_6VUQ.json
 DISK  Reading butcher_6vuq/protein.pdb
 DISK  Writing _butcher_protein.pdb
 DISK  Reading butcher_6vuq/_protein_hull.ply
 DISK  Reading butcher_6vuq/_protein_mesh.ply
 DISK  Reading butcher_6vuq/pocket_adenine (A).ply
 DISK  Reading butcher_6vuq/pocket_ribose (A).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (A).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (A).ply
 DISK  Reading butcher_6vuq/pocket_arginine (A).ply
 DISK  Reading butcher_6vuq/pocket_adenine (B).ply
 DISK  Reading butcher_6vuq/pocket_ribose (B).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (B).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (B).ply
 DISK  Reading butcher_6vuq/pocket_arginine (B).ply
 DISK  Reading butcher_6vuq/pocket_adenine (C).ply
 DISK  Reading butcher_6vuq/pocket_ribose (C).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (C).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 320/322 [06:28<00:02,  1.14s/it]

/Users/tfb64483/Data/CHIKV_Mac/aligned_files/cx1103b/cx1103b_hippo.pdb
 DISK  Reading butcher_6vuq/butcher_6VUQ.json
 DISK  Reading butcher_6vuq/protein.pdb
 DISK  Writing _butcher_protein.pdb
 DISK  Reading butcher_6vuq/_protein_hull.ply
 DISK  Reading butcher_6vuq/_protein_mesh.ply
 DISK  Reading butcher_6vuq/pocket_adenine (A).ply
 DISK  Reading butcher_6vuq/pocket_ribose (A).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (A).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (A).ply
 DISK  Reading butcher_6vuq/pocket_arginine (A).ply
 DISK  Reading butcher_6vuq/pocket_adenine (B).ply
 DISK  Reading butcher_6vuq/pocket_ribose (B).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (B).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (B).ply
 DISK  Reading butcher_6vuq/pocket_arginine (B).ply
 DISK  Reading butcher_6vuq/pocket_adenine (C).ply
 DISK  Reading butcher_6vuq/pocket_ribose (C).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (C).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 321/322 [06:29<00:01,  1.22s/it]

/Users/tfb64483/Data/CHIKV_Mac/aligned_files/cx0543b/cx0543b_hippo.pdb
 DISK  Reading butcher_6vuq/butcher_6VUQ.json
 DISK  Reading butcher_6vuq/protein.pdb
 DISK  Writing _butcher_protein.pdb
 DISK  Reading butcher_6vuq/_protein_hull.ply
 DISK  Reading butcher_6vuq/_protein_mesh.ply
 DISK  Reading butcher_6vuq/pocket_adenine (A).ply
 DISK  Reading butcher_6vuq/pocket_ribose (A).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (A).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (A).ply
 DISK  Reading butcher_6vuq/pocket_arginine (A).ply
 DISK  Reading butcher_6vuq/pocket_adenine (B).ply
 DISK  Reading butcher_6vuq/pocket_ribose (B).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (B).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (B).ply
 DISK  Reading butcher_6vuq/pocket_arginine (B).ply
 DISK  Reading butcher_6vuq/pocket_adenine (C).ply
 DISK  Reading butcher_6vuq/pocket_ribose (C).ply
 DISK  Reading butcher_6vuq/pocket_phosphate (C).ply
 DISK  Reading butcher_6vuq/pocket_oxyanion (

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 322/322 [06:31<00:00,  1.21s/it]


In [12]:
print('FINISHED')

FINISHED


## Calculate fingerprints

## Plotting

In [14]:
logger.var('Fragalysis ribose', len(animal.poses(tag='[Other] ribose site').compounds))
logger.var('PoseButcher ribose', len(animal.poses(tag='butcher: ribose').compounds))

logger.var('Fragalysis adenine', len(animal.poses(tag='[Other] adenine site').compounds))
logger.var('PoseButcher adenine', len(animal.poses(tag='butcher: adenine').compounds))

logger.var('Fragalysis phosphate', len(animal.poses(tag='[Other] phosphate-binding site').compounds))
logger.var('PoseButcher phosphate', len(animal.poses(tag='butcher: phosphate').compounds))

logger.var('Fragalysis oxyanion', len(animal.poses(tag='[Other] oxyanion hole').compounds))
logger.var('PoseButcher oxyanion', len(animal.poses(tag='butcher: oxyanion').compounds))

Fragalysis ribose = 11
PoseButcher ribose = 15
Fragalysis adenine = 57
PoseButcher adenine = 62
Fragalysis phosphate = 9
PoseButcher phosphate = 15
Fragalysis oxyanion = 22
PoseButcher oxyanion = 33


In [15]:
not_ribose = animal.poses(tag='hits').get_by_tag('butcher: ribose', inverse=True)

In [16]:
not_ribose = [303]
for pose in animal.poses.get_by_tag('hits'):
    tags = pose.tags
    
    if 'butcher: ribose' in tags:
        continue
        
    if not pose.name.endswith('a'):
        continue
        
    not_ribose.append(pose.id)

In [17]:
not_ribose = [ i for i in not_ribose if i not in [21, 34, 50, 54, 72, 192, 274]]

In [18]:
selection = animal.poses[not_ribose]

In [21]:
selection.write_sdf('selected_hits.sdf')

Index(['id', 'inchikey', 'alias', 'name', 'smiles', 'path', 'mol', 'compound',
       'target', 'tags', 'inspirations', 'reference', 'observation_longname',
       'ConformerSites', 'CanonSites', 'CrystalformSites', 'Quatassemblies',
       'Crystalforms'],
      dtype='object')
 DISK  Writing selected_hits.sdf


In [22]:
animal.poses['cx1454a'].summary()

P292: cx1454a
inchikey = SEOZHXRTVJPQPZ-UHFFFAOYSA-N
alias = cx1454a
smiles = Cc1nc(N)ccc1Br
compound = C67 "SEOZHXRTVJPQPZ-UHFFFAOYSA-N"
path = /Users/tfb64483/Data/CHIKV_Mac/aligned_files/cx1454a/cx1454a_hippo.pdb
target = T1 "CHIKV_Mac"
reference = None
tags = ['hits', 'butcher: adenine', '[Other] adenine site']
num_heavy_atoms = 9
metadata = {'observation_longname': 'CHIKV_MacB-x1454_A_401_CHIKV_MacB-x0300+A+401+1', 'ConformerSites': '2a - CHIKV_MacB-x0300', 'CanonSites': '2 - ADP-ribose binding site and subsites', 'CrystalformSites': 'F1a - P31-4fold-NCS - CHIKV_MacB-x0270/A/304/1', 'Quatassemblies': 'A1 - monomer', 'Crystalforms': 'F1 - P31-4fold-NCS', 'exports': ['/Users/tfb64483/Data/CHIKV_Mac/hippo/selected_hits.sdf']}


In [23]:
sys = mp.parse(animal.poses['cx0300a'].path)

parsing /Users/tfb64483/Data/CHIKV_Mac/aligned_files/cx0300a/cx0300a.pdb ... Done.


In [24]:
protein = sys.protein_system

In [25]:
protein.write('template.pdb')

writing template.pdb ...                  Done.


In [35]:
animal.db.close()

DEBUG: hippo3.Database.close()
Closed connection to CHIKV_master.sqlite
